# Geekbench benchmark on Android

Geekbench4 is an app offering several benchmarks to run on android smartphones. The one used in this notebook is the '**CPU**' benchmark, which runs several workloads that follow the lines of what is commonly run by smartphones (AES, JPEG codec, FFT, and so on). The benchmark runs all the tests in '**Single-Core**' mode as well as in '**Multi-Core**' in order to compare the single-thread and multi-thread performances of the device.

**Do note that the benchmark will attempt to upload its results, which includes some hardware information**

In [1]:
from conf import LisaLogging
LisaLogging.setup()

2017-04-04 16:07:16,086 INFO    : root         : Using LISA logging configuration:
2017-04-04 16:07:16,183 INFO    : root         :   /home/valsch01/Work/lisa/logging.conf


In [2]:
%pylab inline

import json
import os

# Support to access the remote target
import devlib
from env import TestEnv

# Import support for Android devices
from android import Screen, Workload

# Support for trace events analysis
from trace import Trace

# Suport for FTrace events parsing and visualization
import trappy

import pandas as pd

Populating the interactive namespace from numpy and matplotlib


## Support Functions

This function helps us run our experiments:

In [3]:
def experiment():
    
    # Configure governor
    target.cpufreq.set_all_governors('sched')
    
    # Get workload
    wload = Workload.getInstance(te, 'Geekbench')
    
    # Run Geekbench workload
    wload.run(te.res_dir, test_name='CPU', collect='ftrace')
        
    # Dump platform descriptor
    te.platform_dump(te.res_dir)

## Test environment setup
For more details on this please check out **examples/utils/testenv_example.ipynb**.

**devlib** requires the ANDROID_HOME environment variable configured to point to your local installation of the Android SDK. If you have not this variable configured in the shell used to start the notebook server, you need to run a cell to define where your Android SDK is installed or specify the ANDROID_HOME in your target configuration.

In case more than one Android device are conencted to the host, you must specify the ID of the device you want to target in **my_target_conf**. Run **adb devices** on your host to get the ID.

In [4]:
# Setup target configuration
my_conf = {

    # Target platform and board
    "platform"     : 'android',
    "board"        : 'pixel',
    
    # Device
    "device"       : "HT67M0300128",
    
    # Android home
    "ANDROID_HOME" : "/home/vagrant/lisa/tools/android-sdk-linux/",

    # Folder where all the results will be collected
    "results_dir" : "Geekbench_example",

    # Define devlib modules to load
    "modules"     : [
        'cpufreq'       # enable CPUFreq support
    ],

    # FTrace events to collect for all the tests configuration which have
    # the "ftrace" flag enabled
    "ftrace"  : {
         "events" : [
            "sched_switch",
            "sched_wakeup",
            "sched_wakeup_new",
            "sched_overutilized",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "cpu_capacity",
            "cpu_frequency",
         ],
         "buffsize" : 100 * 1024,
    },

    # Tools required by the experiments
    "tools"   : [ 'trace-cmd', 'taskset'],
}

In [ ]:
# Initialize a test environment using:
te = TestEnv(my_conf, wipe=False)
target = te.target

## Workloads execution

This is done using the **experiment** helper function defined above which is configured to run a **Geekbench - CPU** experiment.

In [ ]:
# Initialize Workloads for this test environment
results = experiment()

## Results collection

Geekbench4 uses a baseline score of 4000, which is the benchmark score of an Intel Core i7-6600U. Higher scores are better, with double the score indicating double the performance. You can have a look at the results for several android phones here https://browser.primatelabs.com/android-benchmarks

In [37]:
class TE():
    res_dir="/home/valsch01/Work/lisa/results/hikey_geekbench_eas"
    
te = TE()

In [112]:
compare_dirs = [ "/home/valsch01/Work/lisa/ipynb/scratchpad/geekbench4/sched" ]

compare_geekbenches = []
for d in compare_dirs:
    files = [f for f in os.listdir(d) if f.endswith(".gb4")]
    for f in files:
        compare_geekbenches.append(get_geekbench(d + "/" + f))
        
print get_global_results(compare_geekbenches[0])

              Single-Core  Multi-Core
Global score         1702        3977


In [192]:
def get_geekbench_name(geekbench):
    gov = ""
    build = ""
    for metric in geekbench["metrics"]:
        if metric["name"] == "Governor":
            gov = metric["value"]
        elif metric["name"] == "Build":
            build = metric["value"]
    
    return "Build: {} - Gov: {}".format(build, gov)

def get_geekbench_benchmarks(geekbench):
    #return [section for section in geekbench["sections"]]
    res = []
    for section in geekbench["sections"]:
        res.append(section)
    return res
    
def get_geekbench_workloads(geekbench):
    #return [benchmark["workloads"] for benchmark in get_geekbench_benchmarks(geekbench)]
    res = []
    for benchmark in get_geekbench_benchmarks(geekbench):
        for workload in benchmark["workloads"]:
            res.append(workload)
    return res

In [218]:
def get_geekbench(filepath):
    res = {}
    with open(filepath) as fd:
        res = json.loads(fd.read()) 
    return res

def get_global_results(geekbench):
    data = []
    idx = []
    for section in geekbench["sections"]:
        data.append(section["score"])
        idx.append(section["name"])
    df = pd.DataFrame(data=data, index=idx, columns=["Global score"])
    return df.T

def get_workloads_results(geekbench):    
    data = []
    idx = []
    benchmark_fields = ["score", "threads", "runtime_mean", "rate_string"]

    for section in geekbench["sections"]:
        for workload in section["workloads"]:
            wl_data = []
            for field in benchmark_fields:
                wl_data.append(workload[field])            
            data.append(tuple(wl_data))
            idx.append( (section["name"], workload["name"]) )  

    sections = [section["name"] for section in geekbench["sections"]]
    workloads = set([workload["name"] for section in geekbench["sections"] for workload in section["workloads"]]) 
    idx = pd.MultiIndex.from_tuples(idx)
        
    df = pd.DataFrame(data, index=idx, columns=benchmark_fields).unstack(0)
    df.columns = df.columns.swaplevel(0,1)
    df.sortlevel(0, axis = 1, inplace=True)
    return df

# compare one geekbench with another (must be of same type)
def compare_results(geekbench, comparison=None):   
    comparison = compare_geekbenches[0]
    bench_types = [benchmark["name"] for benchmark in get_geekbench_benchmarks(geekbench)]
    
    print "Comparing your {} with {}".format(get_geekbench_name(geekbench), get_geekbench_name(comparison))
    
    for bench_type in bench_types:
        # Could be Compute, Multi-Core or Single-Core
        other_res = get_workloads_results(comparison)
        other_res = other_res[bench_type]
        my_res = get_workloads_results(geekbench)
        my_res = my_res[bench_type]
        
        columns = ["New score", "Previous score", "Delta"]
        data = []
        for workload in workloads:
            other_score = other_res["score"][workload]
            my_score = my_res["score"][workload]
            delta = (1.0 * my_score - other_score) / other_score
            data.append((my_score, other_score, "{:.2f}%".format(delta * 100)))
        df = pd.DataFrame(data, index=workloads, columns=columns)
        print bench_type
        display(df)
            

In [43]:
compare = []

for directory in to_compare:
    for f in os.listdir(directory):
        if f.endswith(".gb4"):
            res = directory + "/" + f
            
    compare.append([res, get_workloads_results(get_geekbench(res))])

In [219]:
for f in os.listdir(te.res_dir):
    if f.endswith(".gb4"):
        geekbench = get_geekbench(te.res_dir + "/" + f)
        gdf = get_global_results(geekbench)
        wdf = get_workloads_results(geekbench)

        print "---- Global benchmark results ----"
        if "browser" in geekbench:
            print "Uploaded results available here : " + geekbench["browser"]["url"]
        #display(gdf)
        print "---- Detailled benchmark results ----"
        #display(wdf)
        
        compare_results(geekbench)
        
        #multi = wdf["Multi-Core"]["score"]
        #single = wdf["Single-Core"]["score"]

        #workloads = [item[0] for item in multi.iteritems()]
        #for c in compare:
        #    print "---- Comparing with {} ----".format(c[0])
        #    for workload in workloads:
        #        original_score = c[1]["Multi-Core"]["score"][workload]
        #        delta = (multi[workload]*1.0 - original_score) / original_score
        #        print "{} -> {:.4f} % better results".format(workload, delta * 100)

---- Global benchmark results ----
---- Detailled benchmark results ----
Comparing your Build: hikey960-userdebug O NYC 82 test-keys - Gov: sched with Build: sailfishf-userdebug 7.1.1 NMF26P 3525730 dev-keys - Gov: sched
Single-Core


,New score,Previous score,Delta
AES,1176,777,51.35%
Camera,2031,2569,-20.94%
Canny,1663,2151,-22.69%
Dijkstra,2366,2095,12.94%
Face Detection,1759,2080,-15.43%
Gaussian Blur,1348,2312,-41.70%
HDR,2307,2808,-17.84%
HTML5 DOM,2436,292,734.25%
HTML5 Parse,1993,1578,26.30%
Histogram Equalization,1635,1477,10.70%


Multi-Core


,New score,Previous score,Delta
AES,1655,2035,-18.67%
Camera,7240,7115,1.76%
Canny,6246,6722,-7.08%
Dijkstra,8229,4906,67.73%
Face Detection,6120,5347,14.46%
Gaussian Blur,4010,5730,-30.02%
HDR,2644,7351,-64.03%
HTML5 DOM,2836,1419,99.86%
HTML5 Parse,2304,4109,-43.93%
Histogram Equalization,2570,4607,-44.22%
